<a href="https://colab.research.google.com/github/mantoan-thi/Day_trade_mini_indice/blob/main/Bot_metatrade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from talib.abstract import *
import matplotlib.pyplot as plt
import datetime, time
%matplotlib inline

In [ ]:
# estabelecemos a conexão com o terminal MetaTrader 5 para a conta especificada\n",
if not mt5.initialize(login=3002637411, server="Rico-Demo",password="heidegger*"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [ ]:
# obtemos o número de instrumentos financeiros
symbols=mt5.symbols_total()
if symbols>0:
    print("Total symbols =",symbols)
else:
    print("Symbols not found")

Total symbols = 76095


In [ ]:
def balance():
    account_info_dict = mt5.account_info()._asdict()
    df=pd.DataFrame(list(account_info_dict.items()),columns=['property','value'])
    balance = df['value'][10]
    return balance

In [ ]:
# obtemos todos os símbolos
lista = []
count = 0
dados = pd.DataFrame()
symbols=mt5.symbols_get()
for s in symbols:
    count+=1
    print('Ativos analisados:',count,end='\r')
    dados = mt5.copy_rates_from_pos(s.name, mt5.TIMEFRAME_D1, 0, 50)
    dados = pd.DataFrame(dados)
    if len(dados) > 40:
        dados['RSI'] = RSI(dados.close, timeperiod=14)
        dados['M14d'] = SMA(dados.RSI,timeperiod=7)
        Rsi_Maior = dados['M14d'].iloc[-1:].values
        if pd.isna(Rsi_Maior)==False:
            lista.append((s.name,Rsi_Maior[0]))
    if count > 10000:
        break

In [ ]:
lista = pd.DataFrame(lista)
lista.rename(columns={0: 'Ativo',1:'RSI'}, inplace = True)
lista.sort_values(by=['RSI'],ascending=False,).head(10)

,Ativo,RSI
274,VALEK30,96.312495
290,VALEL2,94.701421
268,VALEF550,89.375330
534,FHER3,84.283318
5,BBASG24,83.818300
178,PETRM19,81.990155
266,VALEF510,81.922840
648,MOAR3,81.803486
640,MGEL4,81.747570
710,RANI3,81.252005


In [ ]:
dados = []
for sm in symbols:
    sm.name
    dados.append((sm.name,sm.basis,sm.currency_base,sm.option_strike,sm.isin))

In [ ]:
dados = pd.DataFrame(dados)
dados.head(10)

,0,1,2,3,4
0,ABEVA207,ABEV3,BRL,20.25,BRABEV3A0577
1,ABEVA212,ABEV3,BRL,20.75,BRABEV3A0585
2,ABEVA216,ABEV3,BRL,21.25,BRABEV3A05B7
3,ABEVA219,ABEV3,BRL,21.50,BRABEV3A05A9
4,ABEVA221,ABEV3,BRL,21.75,BRABEV3A0593
5,ABEVB200,ABEV3,BRL,19.50,BRABEV3B0568
6,ABEVB215,ABEV3,BRL,21.00,BRABEV3B0550
7,ABEVB247,ABEV3,BRL,24.25,BRABEV3B0543
8,ABEVC196,ABEV3,BRL,19.25,BRABEV3C0526
9,ABEVC222,ABEV3,BRL,21.75,BRABEV3C04Y8


In [ ]:
symbols[77751]

SymbolInfo(custom=False, chart_mode=1, select=False, visible=False, session_deals=0, session_buy_orders=0, session_sell_orders=0, volume=0, volumehigh=0, volumelow=0, time=0, digits=2, spread=0, spread_float=True, ticks_bookdepth=32, trade_calc_mode=35, trade_mode=4, start_time=0, expiration_time=1637355599, trade_stops_level=0, trade_freeze_level=0, trade_exemode=3, swap_mode=0, swap_rollover3days=3, margin_hedged_use_leg=False, expiration_mode=11, filling_mode=3, order_mode=127, order_gtc_mode=0, option_mode=0, option_right=1, bid=0.0, bidhigh=0.0, bidlow=0.0, ask=0.0, askhigh=0.0, asklow=0.0, last=0.0, lasthigh=0.0, lastlow=0.0, volume_real=0.0, volumehigh_real=0.0, volumelow_real=0.0, option_strike=142.0, point=0.01, trade_tick_value=0.01, trade_tick_value_profit=0.01, trade_tick_value_loss=0.01, trade_tick_size=0.01, trade_contract_size=1.0, trade_accrued_interest=0.0, trade_face_value=0.0, trade_liquidity_rate=0.0, volume_min=100.0, volume_max=52844748.0, volume_step=100.0, volum

In [ ]:
print('Ações para compra: ')
print(lista[lista['RSI']>70].sort_values(by=['RSI'],ascending=False,).head(10))
print('\nAções para venda: ')
print(lista[lista['RSI']<30].sort_values(by=['RSI'],ascending=False,).head(10))

Ações para compra: 
        Ativo        RSI
274   VALEK30  96.312495
290    VALEL2  94.701421
268  VALEF550  89.375330
534     FHER3  84.283318
5     BBASG24  83.818300
178   PETRM19  81.990155
266  VALEF510  81.922840
648     MOAR3  81.803486
640     MGEL4  81.747570
710     RANI3  81.252005

Ações para venda: 
        Ativo        RSI
35   BBASX319  29.919342
7    BBASI352  29.915875
308  VALEW468  29.888266
457     CPLE3  29.753657
139  PETRI247  29.651971
140  PETRI274  29.466671
621     LIGT3  29.324384
192  PETRS157  29.038738
8    BBASI381  29.020040
200   PETRS20  28.932099


In [ ]:
def pegar_dados():
    # solicitamos 10 barras de GBPUSD D1 do dia atual
    par = "ITUB3"
    rates = mt5.copy_rates_from_pos(par, mt5.TIMEFRAME_M5, 0, 2000)
    dados = pd.DataFrame(rates)
    # convertemos o tempo em segundos no formato datetime
    #dados['time']=pd.to_datetime(dados['time'], unit='s')
    #dados['Tag']= (dados.close.shift(-1)-dados.open.shift(-1))+(dados.close.shift(-2)-dados.open.shift(-2))
    #dados['var']= dados['close']/dados['close'].shift(2)-1 
    #dados['Media'] = SMA(dados['var'].shift(1), timeperiod=8)
    dados.to_csv('C:/Users/Thiago/Desktop/arquivos/Dataset/EURUSD_M5.csv')
    return dados
    

In [ ]:
li

In [ ]:
pegar_dados()

,time,open,high,low,close,tick_volume,spread,real_volume
0,1617878400,24.11,24.14,24.08,24.11,15,1,4800
1,1617878700,24.09,24.11,24.08,24.10,24,1,4700
2,1617879000,24.10,24.16,24.09,24.16,14,1,3100
3,1617879300,24.16,24.17,24.14,24.17,8,1,2500
4,1617879600,24.17,24.17,24.12,24.12,10,1,4500
...,...,...,...,...,...,...,...,...
1995,1620837300,24.36,24.39,24.34,24.39,16,1,4000
1996,1620837600,24.40,24.40,24.35,24.37,21,1,3300
1997,1620837900,24.38,24.43,24.38,24.43,19,1,4400
1998,1620838200,24.42,24.44,24.40,24.41,24,1,5600


In [ ]:
def testar(Cp_menor,Cp_maior,Vd_menor,Vd_maior,payout):
    
        dados = pegar_dados()
        entrada = 0.01*1000
        ganhos = payout * entrada
    
        # Compra
        dados.loc[(dados['Media']> Cp_menor) & (dados['Media']< Cp_maior), 'Compra'] = 1
        # venda
        dados.loc[(dados['Media']> Vd_menor) & (dados['Media']< Vd_maior), 'Venda'] = 1
    
        dados.loc[(dados['Compra'] == 1) & (dados['var'] >= 0),'acerto_compras'] = ganhos
        dados.loc[(dados['Compra'] == 1) & (dados['var'] <= 0),'acerto_compras'] = -(entrada)
    
        dados.loc[(dados['Venda'] == 1) & (dados['var'] <= 0),'acerto_vendas'] = ganhos
        dados.loc[(dados['Venda'] == 1) & (dados['var'] >= 0),'acerto_vendas'] = -(entrada)
    
        dados.loc[(dados['acerto_compras'].isnull()),'acerto_compras'] = 0
        dados.loc[(dados['acerto_vendas'].isnull()),'acerto_vendas'] = 0  
        dados['Total'] = dados['acerto_compras']+ dados['acerto_vendas']
    
        dados['acumulado'] = dados['Total'].cumsum()
        win = 0
        loss = 0
        ganhos_compras = 0
        perdas_compras = 0
    
        win1 = 0
        loss1 = 0
        ganho_vendas = 0
        perdas_vendas = 0
        for i in dados['acerto_compras']:
            if i > 0:
                win +=1
                ganhos_compras+=i
            
            if i < 0:
                loss +=1
                perdas_compras+=i
    
        for x in dados['acerto_vendas']:
            if x > 0:
                win1 +=1
                ganho_vendas+=x
            
            if x < 0:
                loss1 +=1
                perdas_vendas+=x
    
        win_total = win+win1
        loss_total = loss+loss1
        ganhos = ganhos_compras+ganho_vendas
        perdas = -(perdas_compras+perdas_vendas)
        
        #Drawdown
        vl_max = max(dados['acumulado'])
        vl_min = min(dados['acumulado'])
        if vl_max == 0 or vl_min == 0:
            draw = 0
        else:
            draw = (vl_max-vl_min)/vl_max
        if ganhos > 0 and perdas > 0:
            ft = round(ganhos/perdas,2)
            if ft > 1.2:
                print('\033[40;1;34m','>>>>>>>>>>>>>>>>>>>>>>>>>> Backtest!! <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<','\033[0;0m')
                print('\nValor da entrada.: $',entrada)
                print('Payout.:','{0:.2f}%'.format(payout),'\\n')
                
                print('Ranger Cp_menor: ',float(Cp_menor))
                print('Ranger Cp_maior: ',float(Cp_maior),'\\n')
                print('Ranger Vd_menor: ',float(Vd_menor))
                print('Ranger Vd_maior: ',float(Vd_maior),'\\n')
    
                print('Quantidade de win.:',win_total)
                print('Quantidade de loss.:',loss_total)
                print('Acertividade: ','{0:.2f}%'.format(win_total/(win_total+loss_total)))
    
                print('\nLucro liquido: $',round(ganhos,3))
                print('Prejuizo liquido: $',round(perdas,3))
                print('Lucro: $',round(ganhos-perdas,3))
                print('Lucro médio por operação: $',round(win_total/(win_total+loss_total),2))
    
                print('\n% Ganhos: ','{0:.2f}%'.format(win_total/(win_total+loss_total)))
                print('Fator de lucro:',round(ganhos/perdas,2))
                print('\nDrawdown: ','{0:.2f}%'.format(draw))
                print('\nRebaixamento: ',vl_min)
                print('n/Saldo em conta: ',balance())
                plt.plot(dados['acumulado'])
                plt.show()
                print('\033[40;1;34m','>>>>>>>>>>>>>>>>>>>>>>>>>> FIM <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<','\033[0;0m')
            #else:
                #print(\Ranger menor: \,menor)
                #print(\Ranger maior: \,maior)
                #print('\\n% Ganhos: ','{0:.2f}%'.format(win_total/(win_total+loss_total)))
                #print('Fator de lucro:',round(ganhos/perdas,2))
                #print('\\nDrawdown: ','{0:.2f}%'.format(draw))
                #print('\\033[40;1;34m','>>>>>>>>>>>>>>>>>>>>>>>>>> FIM!! <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<','\\033[0;0m')
        else:
            ft = 0
        return ft

In [ ]:
payout = 0.80
Cp_menor = 0.0001
Cp_maior = 0.0005
Vd_menor = -0.0005
Vd_maior = -0.0003
ft = testar(Cp_menor,Cp_maior,Vd_menor,Vd_maior,payout)

In [ ]:
lista=[]
for i in range(1,10):
    for x in range(i+1,10):
        Cp_menor = float(i/100000)
        Cp_maior = float(x/100000)
        Vd_menor = -float(x/100000)
        Vd_maior = -float(i/100000)

        ft = testar(Cp_menor,Cp_maior,Vd_menor,Vd_maior,payout)
        lista.append([payout,Cp_menor,Cp_maior,Vd_menor,Vd_maior,ft])
        lista2 = pd.DataFrame(lista)
lista3 = lista2.rename({0: 'Payout',1: 'Cp_menor',2: 'Cp_maior',3: 'Vd_menor',4: 'Vd_maior',5:'Fator_Lucro'},axis=1)  # new method\n"

In [ ]:
count =0
c = 1.2
for b in lista3['Fator_Lucro']:
    if b > 1:
        if b > c:
            c = b
            #print('\\033[40;1;34m','>>>>>> Fator de lucro!! <<<<<<<','\\033[0;0m')
            #print('\\nMaior fator de lucro.: ',lista3['Fator_Lucro'].loc[count:count].values,'\\n'),
            #print('Ativo: ',par,'\\n')
            #print(lista3.loc[count:count],'\\n\\n')   
            #print('\\033[40;1;34m','>>>>>> FIM!! <<<<<<<','\\033[0;0m\\n')
            new_count = count
    count+=1
print(lista3.loc[new_count:new_count],'\n\n')

In [ ]:
def melhor_ranger():
    count =0
    new_count = 0
    c = 1.2
    for b in lista3['Fator_Lucro']:
        if b > 1:
            if b > c:
                c = b
                #print('\\033[40;1;34m','>>>>>> Fator de lucro!! <<<<<<<','\\033[0;0m')
                #print('\\nMaior fator de lucro.: ',lista3['Fator_Lucro'].loc[count:count].values,'\\n'),
                #print('Ativo: ',par,'\\n')
                #print(lista3.loc[count:count],'\\n\\n')   
                #print('\\033[40;1;34m','>>>>>> FIM!! <<<<<<<','\\033[0;0m\\n')
                new_count = count
        count+=1
    range = lista3.loc[new_count:new_count]
    Cp_menor = round(max(range.Cp_menor),4)
    Cp_maior = max(range.Cp_menor)
    Vd_menor = max(range.Vd_menor)
    Vd_maior = max(range.Vd_maior)
    #print(lista3.loc[new_count:new_count],'\\n\\n') 
    return Cp_menor,Cp_maior,Vd_menor,Vd_maior

In [ ]:
def sinal():
    while True:
        dados = pegar_dados()
        Cp_menor,Cp_maior,Vd_menor,Vd_maior = melhor_ranger()
        if dados['Media'].iloc[-1] > Cp_menor and dados['Media'].iloc[-1] < Cp_maior:
            sinal = 1 # Comprar 
            break
        else:
            if dados['Media'].iloc[-1] > Vd_menor and dados['Media'].iloc[-1] < Vd_maior:
                sinal = -1 # Comprar 
                break
            else:
                print('Ficar fora do mercado!',end='\r')
    return sinal

In [ ]:
def Validar_ativo(): 
    symbol_info = mt5.symbol_info(par)
    if symbol_info is None:
        print(symbol, "Não encontrato")
        mt5.shutdown()
        quit()
    else:
        return True

In [ ]:
def Adicionar_ativo_grafico():
    symbol = par
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(symbol, "not found, can not call order_check()")
        mt5.shutdown()
        quit()
#adicionado symbol se nao existir
    if not symbol_info.visible:
        print('Symbol Não visivel, tentnado adicionar')
        if not mt5.symbol_select(symbol,True):
            print('symbol_select({{}})failed, exit', symbol)
            mt5.shutdown()
            quit()
    else:
        return True

In [ ]:
def breakeven_compra(position_id,lote,pr_inicial,tp_comp,sl_comp):
    symbol = par
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).ask
    tp_comp_rev = price + 100 * point
    sl_comp_rev = price - 10 * point
    while True:
        lot = lote
        symbol = par
        #dados = pegar_dados()
        #pr_atual = dados['close'].iloc[-1]
        point = mt5.symbol_info(symbol).point
        price = mt5.symbol_info_tick(symbol).ask
        var = (price/pr_inicial-1)*1000
        tp_comp_rev = price + 100 * point
        sl_comp_rev = price - 10 * point
        if var > 0.12:
            if sl_comp_rev > tp_comp:
                tp_comp = tp_comp_rev
                sl_comp = sl_comp_rev
                # Preparando ordem de compra
                desviation = 1
                request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "symbol": symbol,
                    "volume": lot,
                    "type": mt5.ORDER_TYPE_BUY,
                    "price": price,
                    "position": position_id,
                    "sl": sl_comp,
                    "tp": tp_comp,
                    "magic": 234000,
                    "desviation": desviation,
                    "comment": "prython script open",
                    "type_time":mt5.ORDER_TIME_GTC,
                    'type_filling':mt5.ORDER_FILLING_RETURN,
                }
                result = mt5.order_send(request)
                if result.retcode != mt5.TRADE_RETCODE_DONE:
                    print(". 2. order_send failed, retcode={}".format(result.retcode),end='\r') 
        positions_total=mt5.positions_total()
        if positions_total==0:
            break

In [ ]:
def breakeven_venda(position_id,lote,pr_inicial,tp_venda,sl_venda):
    symbol = par
    point = mt5.symbol_info(symbol).point
    price=mt5.symbol_info_tick(symbol).bid
    tp_venda = price - 100 * point
    sl_venda = price + 10 * point
    while True:
        symbol = par
        lot= lote
        #dados = pegar_dados()
        #pr_atual = dados['close'].iloc[-1]
        point = mt5.symbol_info(symbol).point
        price=mt5.symbol_info_tick(symbol).bid
        var = (price/pr_inicial-1)*1000
        tp_venda_rev = price - 100 * point
        sl_venda_rev = price + 10 * point
        if var < -0.12:
            if sl_venda_rev < sl_venda:
                tp_venda = tp_venda_rev
                sl_venda = sl_venda_rev
                #preparando a ordem de venda 
                desviation = 1
                request2={
                    "action": mt5.TRADE_ACTION_SLTP,
                    "symbol": symbol,
                    "volume": lot,
                    "type": mt5.ORDER_TYPE_SELL,
                    "price": price,
                    "position": position_id,
                    "sl": sl_venda,
                    "tp": tp_venda ,
                    "deviation": desviation,
                    "magic": 234000,
                    "comment": "python script close",
                    "type_time": mt5.ORDER_TIME_GTC,
                    "type_filling": mt5.ORDER_FILLING_RETURN,
                }
                result = mt5.order_send(request2)
        positions_total=mt5.positions_total()
        if positions_total==0:
            break

In [ ]:
def Buy_Sell(dir,lote):
    symbol = 'EURUSD'
    lot = lote
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).ask
    pr_inicial = price
    deviation = 1
    tp_comp = price + 100 * point
    sl_comp = price - 100 * point
    if dir == 1:
        # Preparando ordem de compra
        point = mt5.symbol_info(symbol).point
        price = mt5.symbol_info_tick(symbol).ask
        desviation = 1
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot,
            "type": mt5.ORDER_TYPE_BUY,
            "price": price,
            "sl": sl_comp,
            "tp": tp_comp,
            "magic": 234000,
            "desviation": desviation,
            "comment": "prython script open",
            "type_time":mt5.ORDER_TIME_GTC,
            'type_filling':mt5.ORDER_FILLING_RETURN,
        }
        result = mt5.order_send(request)
        #print(result)
        #if result.retcode != mt5.TRADE_RETCODE_DONE:
        #    print(". 2. order_send failed, retcode={}".format(result.retcode),end='\r')  
        position_id=result.order
        breakeven_compra(position_id,lote,price,tp_comp,sl_comp)
    if dir == -1:
        #preparando a ordem de venda 
        point = mt5.symbol_info(symbol).point
        price=mt5.symbol_info_tick(symbol).bid
        desviation = 1
        tp_venda = price - 100 * point
        sl_venda = price + 100 * point
        request2={
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot,
            "type": mt5.ORDER_TYPE_SELL,
            "price": price,
            "sl": sl_venda,
            "tp": tp_venda ,
            "deviation": desviation,
            "magic": 234000,
            "comment": "python script close",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_RETURN,
        }
        result = mt5.order_send(request2)
        print(result)
        position_id=result.order
        breakeven_venda(position_id,lote,price,tp_venda,sl_venda)

In [ ]:
while True:
    if sinal() ==1:
        print('Ordem de compra emitida!',end='\r')
        Buy_Sell(1,0.01)
    if sinal() ==-1:
        print('Ordem de venda emitida!',end='\r')
        Buy_Sell(-1,0.01)